In [1]:
import torch
import random
import time
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import matplotlib.pyplot as plt

from functions import enviar_chute_site


In [2]:
memory = {}
valor_embeddings = {}

# Palavras genéricas para abrir sentidos semânticos
temas_exploratorios = [
    "object", "place", "person", "animal", "emotion",
    "food", "vehicle", "technology", "family", "music",
    "clothing", "tool", "plant", "profession", "color"
]
temas_usados = set()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

def get_embedding(word):
    with torch.no_grad():
        inputs = tokenizer(word, return_tensors="pt")
        outputs = model(**inputs)
        # Usa o embedding do [CLS] token
        embedding = outputs.last_hidden_state[0][0].numpy()
    return embedding

# Sugestão adaptativa
def sugerir_proximo():
    print("\n🎯 Gerando novas sugestões...")

    if not memory:
        sugestao = random.choice(temas_exploratorios)
        temas_usados.add(sugestao)
        print(f"🧭 Começando exploração: {sugestao}")
        return sugestao

    # Classificação refinada por score
    perfeitos = [(p, s) for p, s in memory.items() if s < 50]
    otimos    = [(p, s) for p, s in memory.items() if 50 <= s < 200]
    bons      = [(p, s) for p, s in memory.items() if 200 <= s < 300]
    oks       = [(p, s) for p, s in memory.items() if 300 <= s < 500]
    ruins     = [(p, s) for p, s in memory.items() if s >= 500]

    todos_ruins = len(ruins) == len(memory)

    if todos_ruins:
        restantes = [t for t in temas_exploratorios if t not in temas_usados]
        if restantes:
            sugestao = random.choice(restantes)
            temas_usados.add(sugestao)
            print(f"🔍 Todos os chutes estão ruins. Explorando novo tema: {sugestao}")
            return sugestao
        else:
            print("⚠️ Todos os temas exploratórios já foram usados. Chutando palavra genérica aleatória.")
            return random.choice([w for w in tokenizer.vocab.keys() if w.isalpha() and len(w) > 4 and w not in memory])

    # Refinamento inteligente com prioridade por nível
    base = []

    if len(perfeitos) >= 1:
        base = sorted(perfeitos, key=lambda x: x[1])  # usa todos
        print(f"💥 Refinando a partir de perfeitos ({len(base)}): {[p for p, _ in base]}")
    elif len(otimos) >= 1:
        base = sorted(otimos, key=lambda x: x[1])
        print(f"🌟 Refinando a partir de ótimos ({len(base)}): {[p for p, _ in base]}")

    elif len(bons) >= 1:
        base = sorted(bons, key=lambda x: x[1])
        print(f"👌 Refinando a partir de bons ({len(base)}): {[p for p, _ in base]}")

    elif len(oks) >= 1:
        base = sorted(oks, key=lambda x: x[1])
        print(f"🙂 Refinando a partir de oks ({len(base)}): {[p for p, _ in base]}")
    else:
        base = sorted(memory.items(), key=lambda x: x[1])[:2]
        print(f"🤔 Refinando com os menos ruins: {[p for p, _ in base]}")

    # Cálculo do vetor médio com pesos
    vetores = np.array([get_embedding(p) for p, _ in base])
    pesos = np.array([1 / (s + 1) for _, s in base])
    pesos = pesos / pesos.sum()
    vetor_medio = np.average(vetores, axis=0, weights=pesos)

    # Seleção de candidatos
    candidatos = [w for w in tokenizer.vocab.keys() if w.isalpha() and len(w) > 4 and w not in memory]
    random.shuffle(candidatos)
    candidatos = candidatos[:1000]

    melhor_palavra = None
    melhor_sim = -1
    dic_sim = {}
    for w in candidatos:
        vec = get_embedding(w)
        sim = cosine_similarity([vetor_medio], [vec])[0][0]
        dic_sim[w] = sim

    # Ordena o dic_sim pelo valor de similaridade (do maior para o menor)
    dic_sim_ordenado = sorted(dic_sim.items(), key=lambda x: x[1], reverse=True)
    palavras = dic_sim_ordenado[:10]  # pega as 10 melhores palavras

    print(f"✅ Próxima sugestão: {melhor_palavra} (sim={melhor_sim:.4f})")
    return palavras

def loop_automatico():
    while True:
        proximas = sugerir_proximo()
        for proxima in proximas:
            chute, score = enviar_chute_site(proxima)
        
            if score is not None:
                if chute not in memory:
                    memory[chute] = score
                    print(f"💡 Chute: {chute} (score={score})")
                else:
                    print(f"🔄 Chute repetido: {chute} (score={score})")
            else:
                print("⏭️ Ignorando chute inválido ou repetido.")

            if score <= 10:
                print("🎉 Palavra secreta encontrada:", chute)
                break

In [4]:
# Inicializa navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://contexto.me/en/")


In [ ]:
# lista de palavras manuais a testar 
# palavras_a_testar = ["place", "try", "fix", "link"] 

# for p in palavras_a_testar:
#     valor_embeddings[p] = get_embedding(p)
#     _,memory[p] = enviar_chute_site(driver, p)

In [10]:
proximos = sugerir_proximo()
print(f"Próximas sugestões: {proximos}")


🎯 Gerando novas sugestões...
💥 Refinando a partir de perfeitos (1): ['link']
✅ Próxima sugestão: None (sim=-1.0000)
Próximas sugestões: [('eligibility', np.float64(0.9669995892306966)), ('overview', np.float64(0.9662007320411513)), ('outlets', np.float64(0.9648468379699994)), ('gateway', np.float64(0.9637623669586088)), ('network', np.float64(0.9634852767819001)), ('shared', np.float64(0.9634391294257358)), ('legacy', np.float64(0.9634298208818348)), ('users', np.float64(0.9634072489142185)), ('surfing', np.float64(0.9633091416800056)), ('hometown', np.float64(0.9632329390299588))]


In [ ]:
loop_automatico()